In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

# Read the dataset
df = pd.read_csv('climate_change_data.csv', encoding='latin1')

# Impute null values in specified columns with the mean
columns_to_impute = df.columns[7:]
df[columns_to_impute] = df[columns_to_impute].apply(lambda x: x.fillna(x.mean()))

# Filter data for temperature change element
df = df.loc[df['Element'] == 'Temperature change']

# Replace month ranges with season names
df.replace(to_replace=r'Dec\x96Jan\x96Feb', value='Winter', regex=True, inplace=True)
df.replace(to_replace=r'Mar\x96Apr\x96May', value='Spring', regex=True, inplace=True)
df.replace(to_replace=r'Jun\x96Jul\x96Aug', value='Summer', regex=True, inplace=True)
df.replace(to_replace=r'Sep\x96Oct\x96Nov', value='Fall', regex=True, inplace=True)

# Drop unnecessary columns
df.drop(['Area Code', 'Months Code', 'Element Code', 'Element', 'Unit'], axis=1, inplace=True)
df.rename(columns={'Area': 'country_name', 'Months': 'months'}, inplace=True)

# Melt the DataFrame
df = pd.melt(df, id_vars=['country_name', 'months'], var_name='year', value_name='temp_change')
df['year'] = [i.split('Y')[-1] for i in df.year]

# Convert 'year' column to numeric
df['year'] = pd.to_numeric(df['year'], errors='coerce')

# Select data for the last ten years
df_last_ten_years = df[df['year'].between(2010, 2019, inclusive='both')]

# Group by country and calculate the mean temperature change
df_country_mean = df_last_ten_years.groupby('country_name')['temp_change'].mean().reset_index()

# Select the top 10 countries with the highest mean temperature change
top_10_countries = df_country_mean.nlargest(10, 'temp_change')

# Create subplots using GridSpec
fig = plt.figure(figsize=(18, 18))
gs = GridSpec(2, 2, figure=fig, height_ratios=[1,2], hspace=0.5)

# Plot 1: Top 10 Countries with the Highest Mean Temperature Change
ax1 = fig.add_subplot(gs[0, 0])
bar1 = ax1.barh(top_10_countries['country_name'], top_10_countries['temp_change'], color='purple')
ax1.bar_label(bar1, fmt='%.2f', padding=3)
ax1.set_xlim([0, 4.0])
ax1.invert_yaxis()
ax1.set_title("Top 10 Countries with the Highest Mean Temperature Change in the Last Decade", weight='bold')
ax1.set_xlabel("Mean Temperature Change (°C)")
ax1.set_ylabel("Countries")

# Plot 2: Global Temperature Change Trends
df_world = df[df['country_name'] == 'World']
ax2 = fig.add_subplot(gs[0,1])
ax2.plot(df_world['year'], df_world['temp_change'], '-', color='gold', linewidth=3)
ax2.bar(df_world['year'], df_world['temp_change'], color='gold')
ax2.set_xticks(df_world['year'])
ax2.set_xticklabels(df_world['year'], rotation=90)
ax2.set_title("Global Temperature Change Trends", weight='bold')
ax2.set_xlabel("Years")
ax2.set_ylabel("Temperature Change (°C)")

# Plot 3: Areas with the Lowest Temperature Change
df_last_decade = df[df['year'].between(2010, 2019)]
df2 = df_last_decade.groupby(['country_name']).agg({'temp_change': 'mean'}).reset_index()
df2 = df2.sort_values(by=['temp_change'], ascending=True).head(10)

ax3 = fig.add_subplot(gs[1, 0])
bar2 = ax3.barh(df2['country_name'], df2['temp_change'], color='cyan')
temp = round(df2['temp_change'], 2).to_list()

j = 0
for p in bar2:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    ax3.text(x + width * 1.05, y + height * 0.6, str(temp[j]), ha='center', weight='bold')
    j += 1

ax3.set_xlim([0, 0.8])
ax3.invert_yaxis()
ax3.set_title("Top 10 Areas with Lowest Temperature Change in The Last Decades", weight='bold')
ax3.set_xlabel("Temperature Change (°C)")
ax3.set_ylabel("Countries")

# Plot 4: Temperature Change in 4 Seasons
df6 = df[df['country_name'] == 'World']
winter = df6[df6['months'] == 'Winter']
spring = df6[df6['months'] == 'Spring']
summer = df6[df6['months'] == 'Summer']
fall = df6[df6['months'] == 'Fall']

ax4 = fig.add_subplot(gs[1,1])
ax4.plot(winter['year'], winter['temp_change'], '-', color='blue', label='Winter')
ax4.plot(spring['year'], spring['temp_change'], '-', color='green', label='Spring')
ax4.plot(summer['year'], summer['temp_change'], '-', color='orange', label='Summer')
ax4.plot(fall['year'], fall['temp_change'], '-', color='brown', label='Fall/Autumn')

ax4.set_xticks(df_world['year'])
ax4.set_xticklabels(df_world['year'], rotation=90)
ax4.legend(fontsize='large')
ax4.set_title("The Trend of Temperature Change in 4 Seasons", weight='bold')
ax4.set_xlabel("Years")
ax4.set_ylabel("Temperature Change (°C)")

# Add a common title
fig.suptitle("Climate Change Analysis", fontsize=30, weight='bold')


# Add the three bullet points
bullet_points = [
    "1. Summer has a relatively smooth line, Winter has a more fluctuated line."
    "2. It shows global warming effects are seen mostly in the winter season. While spring also has similar fluctuation to winter, fall has a similar trend to summer.",
    "3. Even if India is a developing country, it also has lots of industrial activities. This list shows that these activities do not have much effect on Temperature rising.",
    "4. All the industrialized countries and the European countries are having most of the climate change.'"
]

# Add bullet points
fig.text(0.5, 0.02, "\n".join(bullet_points), ha='center', va='bottom', fontsize=12, weight='normal')

# Add student name and student ID
student_info = "Student Name: BALA SUBRAMANYAM BATTULA | Student ID: 22075288"
fig.text(0.5, 0.05, student_info, ha='right', va='bottom', fontsize=12, weight='bold')

# Save the resulting plot to a PNG file
# plt.savefig("22075288.png", dpi=300)

# Show plot
plt.show()


